In [3]:
import shutil

def delete_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Folder '{path}' deleted successfully.")
    except OSError as e:
        print(f"Error: {path} : {e.strerror}")

# Example usage:
folder_path = "/notebooks/train/__MACOSX"
delete_folder(folder_path)


Folder '/notebooks/train/__MACOSX' deleted successfully.


In [9]:
import zipfile
import os

# Define the path to the ZIP file
zip_file_path = "/notebooks/train_aug.zip"

# Define the directory where you want to extract the contents
extract_to_dir = "/notebooks/train3/"

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_to_dir):
    os.makedirs(extract_to_dir)

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the extraction directory
    zip_ref.extractall(extract_to_dir)

print("Extraction complete.")


Extraction complete.


In [14]:
import os

path = "/notebooks/train2/images"
num_files = len(os.listdir(path))

print("Number of files in the path:", num_files)


Number of files in the path: 5152


In [18]:
import os

# Define the directories
dir1 = "/notebooks/train/images"
dir2 = "/notebooks/train/labels"

# Get a list of all file names without extension in each directory
files1 = {os.path.splitext(f)[0] for f in os.listdir(dir1) if f.endswith('.png')}
files2 = {os.path.splitext(f)[0] for f in os.listdir(dir2) if f.endswith('.txt')}

# Compare the file names
common_files = files1 & files2
files_only_in_dir1 = files1 - files2
files_only_in_dir2 = files2 - files1

#print("Common files:", common_files)
print("Files only in /notebooks/valid/images:", files_only_in_dir1)
print("Files only in /notebooks/valid/labels:", files_only_in_dir2)

Files only in /notebooks/valid/images: set()
Files only in /notebooks/valid/labels: set()


In [13]:
import os
import shutil

# Define the source and destination directories
src_dir = "/notebooks/train/images"
dst_dir = "/notebooks/valid/images"

# Get a list of all files in the source directory
all_files = os.listdir(src_dir)

# Filter out non-PNG files
png_files = [f for f in all_files if f.endswith('.png')]

# Sort the files to ensure consistency
png_files.sort()

# Get the last 152 files
last_files = png_files[-152:]

# Move the files
for file in last_files:
    src_path = os.path.join(src_dir, file)
    dst_path = os.path.join(dst_dir, file)
    shutil.move(src_path, dst_path)

In [ ]:
import shutil

source_dir = "/notebooks/valid/images"
destination_dir = "/notebooks/train/images"

# Check if destination directory exists
if not os.path.exists(destination_dir):
  os.makedirs(destination_dir)  # Create the directory if it doesn't exist

# Copy all files from source to destination
shutil.copytree(source_dir, destination_dir, copy_function=shutil.copy2)  # Preserve file metadata

print(f"Files copied from {source_dir} to {destination_dir}")


In [1]:
!pip install ultralytics

from IPython import display
display.clear_output()

In [3]:
import yaml
from ultralytics import YOLO

import random
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import pathlib
%matplotlib inline
from collections import Counter

In [3]:
import zipfile

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os
from PIL import Image, ImageEnhance

# Input and output directories
input_dir = "/notebooks/train1/images"
output_dir = "/notebooks/train2/images"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to enhance contrast and saturation
def enhance_image(image_path, output_path, contrast_factor=1.5, color_factor=1.5):
    # Open the image
    image = Image.open(image_path)
    
    # Enhance contrast
    contrast_enhancer = ImageEnhance.Contrast(image)
    image = contrast_enhancer.enhance(contrast_factor)
    
    # Enhance saturation
    color_enhancer = ImageEnhance.Color(image)
    image = color_enhancer.enhance(color_factor)
    
    # Save the enhanced image
    image.save(output_path)

# Process all images in the input directory
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        enhance_image(input_path, output_path)

print("Image enhancement completed.")


Image enhancement completed.


In [4]:
# Load the YAML file
with open('/notebooks/dataset.yaml', 'r') as f:
    data_yaml = yaml.safe_load(f)

# update YAML with absolute path to kaggle data. You must use absolute path, relative path won't work
data_yaml['train'] = '/notebooks/train3/train_aug/images'
data_yaml['val'] = '/notebooks/train3/train_aug/images'

# write to disk
with open('dataset.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

In [5]:
data_yaml

{'names': ['Cấm ngược chiều',
  'Cấm dừng và đỗ',
  'Cấm rẽ',
  'Giới hạn tốc độ',
  'Cấm còn lại',
  'Nguy hiểm',
  'Hiệu lệnh'],
 'nc': 7,
 'path': '..',
 'train': '/notebooks/train3/train_aug/images',
 'val': '/notebooks/train3/train_aug/images'}

In [6]:
baseline_model = YOLO('yolov9c.yaml')

In [7]:
params = {
    'data':'dataset.yaml',
    'batch':4,
    'imgsz':640,
    'epochs':100,
    'optimizer':'Adam',
    'pretrained':True,
    'lr0':0.01,
    'lrf': 0.00001,
    'cos_lr': True,
    'label_smoothing':0.15,
    'hsv_h' : 0.02,
    'hsv_s' : 0.50,
    'hsv_v' : 0.30,
    'degrees' : 5.0,
    'scale' : 0.3,
    'fliplr': 0.30,
    'mosaic': 0.50,
    'plots':True}

In [ ]:
# Training and validation loop
baseline_model.train(**params)

# Save the trained model
torch.save(baseline_model.state_dict(), '/notebooks/save/')

Ultralytics YOLOv8.2.28 🚀 Python-3.9.16 torch-1.12.1+cu116 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov9c.yaml, data=dataset.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train26, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fal

  0%|          | 0.00/22.2M [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    212864  ultralytics.nn.modules.block.RepNCSPELAN4    [128, 256, 128, 64, 1]        
  3                  -1  1    164352  ultralytics.nn.modules.block.ADown           [256, 256]                    
  4                  -1  1    847616  ultralytics.nn.modules.block.RepNCSPELAN4    [256, 512, 256, 128, 1]       
  5                  -1  1    656384  ultralytics.nn.modules.block.ADown           [512, 512]                    
  6                  -1  1   2857472  ultralytics.nn.modules.block.RepNCSPELAN4    [512, 512, 512, 256, 1]       
  7                  -1  1    656384  ultralytics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...

      6/100      3.35G      1.358     0.9548      1.082          8        640: 100%|██████████| 7779/7779 [16:54<00:00,  7.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3890/3890 [05:49<00:00, 11.12it/s]


                   all      31115      55981      0.852      0.697      0.755      0.503

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      7/100      3.35G      1.349     0.9122      1.079          2        640: 100%|██████████| 7779/7779 [16:14<00:00,  7.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3890/3890 [05:50<00:00, 11.11it/s]


                   all      31115      55981      0.882      0.705      0.771      0.514

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      8/100      3.34G      1.332      0.927      1.078         12        640:  22%|██▏       | 1673/7779 [04:02<16:14,  6.27it/s]

In [10]:
import torch
torch.cuda.empty_cache()

In [ ]:
# Load the YAML file
with open('/content/drive/MyDrive/Internship/Code/YOLOv8/annotations/dataset.yaml', 'r') as f:
    data_yaml = yaml.safe_load(f)

# Update YAML with absolute path to kaggle data. You must use absolute path, relative path won't work
data_yaml['train'] = '/content/drive/MyDrive/Internship/Code/YOLOv8/annotations/train/images'
data_yaml['val'] = '/content/drive/MyDrive/Internship/Code/YOLOv8/annotations/train/images'

# Write to disk
with open('dataset.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

# Load the data
data = LoadImagesAndLabels('dataset.yaml')

# Create the model
model = YOLO('yolov8x.yaml')

# Train the model
model.train(data=data, imgsz=640, epochs=100, optimizer='Adam', pretrained=True, lr0=0.01, lrf=0.00001, cos_lr=True, label_smoothing=0.15, hsv_h=0.02, hsv_s=0.50, hsv_v=0.30, degrees=5.0, scale=0.3, fliplr=0.30, mosaic=0.50, plots=True)

# Save the trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/Internship/Code/YOLOv8')

In [ ]:

post_training_files_path = '/content/runs/detect/train'

In [ ]:
with zipfile.ZipFile('post_training_files.zip', 'w') as zip_ref:
    zip_ref.write('/content/runs/detect/train', arcname='train')

In [ ]:
from google.colab import files
files.download("/content/post_training_files.zip")

In [ ]:
def plot_learning_curve(df, train_loss_col, val_loss_col, title):
    plt.figure(figsize=(12, 5))
    sns.lineplot(data=df, x='epoch', y=train_loss_col, label='Train Loss', color='#141140', linestyle='-', linewidth=2)
    sns.lineplot(data=df, x='epoch', y=val_loss_col, label='Validation Loss', color='orangered', linestyle='--', linewidth=2)
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
confusion_matrix_path = os.path.join(post_training_files_path, 'confusion_matrix_normalized.png')

# Read the image using cv2
cm_img = cv2.imread(confusion_matrix_path)

# Convert the image from BGR to RGB color space for accurate color representation with matplotlib
cm_img = cv2.cvtColor(cm_img, cv2.COLOR_BGR2RGB)

# Display the image
plt.figure(figsize=(10, 10), dpi=120)
plt.imshow(cm_img)
plt.axis('off')
plt.show()


In [ ]:
metrics_df = pd.DataFrame.from_dict(metrics.results_dict, orient='index', columns=['Metric Value'])

# Display the DataFrame
metrics_df.round(3)

In [ ]:
valid_images_path = '/content/drive/MyDrive/ML in Medicine/Final/valid/images'

# List all jpg images in the directory
image_files = [file for file in os.listdir(valid_images_path) if file.endswith('.jpg')]

# Select 9 images at equal intervals
num_images = len(image_files)
selected_images = [image_files[i] for i in range(0, num_images, num_images // 9)]

# Initialize the subplot
fig, axes = plt.subplots(3, 3, figsize=(20, 21))
fig.suptitle('Validation Set Inferences', fontsize=24)

# Perform inference on each selected image and display it
for i, ax in enumerate(axes.flatten()):
    image_path = os.path.join(valid_images_path, selected_images[i])
    results = best_model.predict(source=image_path, imgsz=640, conf=0.5)
    annotated_image = results[0].plot(line_width=1)
    annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    ax.imshow(annotated_image_rgb)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
sample_image_path = '/content/drive/MyDrive/ML in Medicine/Final/test/images/BloodImage_00112_jpg.rf.978cec39235980055c2ad7ff8b6f1912.jpg'

# Perform inference on the provided image using best model
results = best_model.predict(source=sample_image_path, imgsz=640, conf=0.7)

# Annotate and convert image to numpy array
sample_image = results[0].plot(line_width=2)

# Convert the color of the image from BGR to RGB for correct color representation in matplotlib
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)

# Display annotated image
plt.figure(figsize=(20,15))
plt.imshow(sample_image)
plt.title('Detected Objects in Sample Image by the Fine-tuned YOLOv8 Model', fontsize=20)
plt.axis('off')
plt.show()

In [ ]:
model=YOLO('/content/drive/MyDrive/ML in Medicine/runs/detect/train/weights/best.pt')

In [ ]:
image_path='/content/drive/MyDrive/ML in Medicine/Final/test/images/BloodImage_00265_jpg.rf.7d102d5f38caddb90a465912fb7bdc5c.jpg'

In [ ]:
image=cv2.imread(image_path)
image

In [ ]:
result=model(image,show=True,save=True)


In [ ]:
import pathlib
import random
import cv2
import matplotlib.pyplot as plt

# Define the directory paths for images and labels
images_dir_test = '/content/drive/MyDrive/ML in Medicine/Final/test/images/BloodImage_00265_jpg.rf.7d102d5f38caddb90a465912fb7bdc5c.jpg'
labels_dir_test = '/content/drive/MyDrive/ML in Medicine/Final/test/labels/BloodImage_00265_jpg.rf.7d102d5f38caddb90a465912fb7bdc5c.txt'

# Load the image using OpenCV's imread function
image = cv2.imread(images_dir_test)

# Get the list of class names from the 'data' dictionary
class_list = data_yaml['names']

# Define a list of colors to be used to draw bounding boxes
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]

# Get the height and width of the image
height, width, _ = image.shape

# Open the label file 'demo_label' in read mode and process each line
with open(labels_dir_test, "r") as file1:
    for line in file1.readlines():
        # Split the line into a list of strings
        split = line.split(" ")

        # Get the class id from the first element of the split list
        class_id = int(split[0])

        # Get the color corresponding to the class id from the 'colors' list
        color = colors[class_id]
        clazz = class_list[class_id]

        # Get the x, y, w, h bounding box coordinates from the split list
        x, y, w, h = float(split[1]), float(split[2]), float(split[3]), float(split[4])

        # Rescale the x, y, w, h values to the size of the image
        box = [int((x - 0.5 * w) * width), int((y - 0.5 * h) * height), int(w * width), int(h * height)]

        # Draw a rectangle on the image using the 'box' and 'color' values
        cv2.rectangle(image, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color, 2)

        # Draw a filled rectangle for the class label on the image
        cv2.rectangle(image, (box[0], box[1] - 20), (box[0] + box[2], box[1]), color, -1)

        # Write the class label on the image
        cv2.putText(image, class_list[class_id], (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 0))

# Show the image using matplotlib
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# Optionally resize the image (commented out in code)
# image = cv2.resize(image, (600, 600))
plt.show()
